In [55]:
# DATA_PATH = '../input/'

# 注意填入你的路径
# DATA_PATH = './'
# DATA_PATH = '../input/shopee-product-matching/'
DATA_PATH = '../input/'

# import psutil

### how to calculate $ F1 $ score
令$ a=TP $ , $ c=FP $ , $ d=FN $ <p>
$ F1=\frac{(2*(a/(a+c))*(a/(a+d)))}{(a/(a+c)+a/(a+d))}=\frac{(2*a)}{(a+c+a+d)} $<p>
其中$ n=a $ , $ len(row.target)=a+d $ , $ len(row[col])=a+c $ <p>
即 $ F1 $ 是精确率和召回率的调和平均数

In [56]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2, matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import tqdm
import gc

# import cudf, cuml, cupy
# from cuml.feature_extraction.text import TfidfVectorizer
# from cuml.neighbors import NearestNeighbors

# 对于样本 row，根据表现分组 (P/N)row[col] 和实际分组 (T/F)row.target 计算 F1
def getMetric(col):
    def f1score(row):
        # 查找两个数组的交集 T and P 
        n = len( np.intersect1d(row.target,row[col]) ) 
        # row.target实际P, row[col]表现P,计算公式见上一个cell
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

In [57]:
COMPUTE_CV = True

test = pd.read_csv(DATA_PATH + 'test.csv')
if len(test)>3: COMPUTE_CV = False # 私榜提交test
else: print('this submission notebook will compute CV score, but commit notebook will not') # 公榜test，因为test.csv只有3行

# COMPUTE_CV = False

if COMPUTE_CV:
    train = pd.read_csv(DATA_PATH + 'train_min.csv')
    train['image'] = DATA_PATH + 'train_images/' + train['image']
    # 对tmp分组，并对每一组提取posting_id的集合
    tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
    # print(tmp)
    # 利用 tmp 将列出同一组的所有 posting_id
    # target 即是 训练集的正确分组
    train['target'] = train.label_group.map(tmp)
else:
    train = pd.read_csv(DATA_PATH + 'test.csv')
    train['image'] = DATA_PATH + 'test_images/' + train['image']
    
print('train shape is', train.shape )
train.head()

this submission notebook will compute CV score, but commit notebook will not
train shape is (16, 6)


,posting_id,image,image_phash,title,label_group,target
2,train_2288590299,../input/train_images/000a190fdd715a2a36faed16...,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,[train_2288590299]
10,train_1598329973,../input/train_images/001d7f5d9a2fac714f4d5f37...,bec8d09693634b4b,Atasan Rajut Wanita LISDIA SWEATER,2462407944,[train_1598329973]
14,train_4287573913,../input/train_images/001f5580b058c6b8e3313219...,dc85e1750687f932,Charger VIZZ VZ-TC11 / batok charger vizz 1A O...,1932232224,[train_4287573913]
15,train_4196427721,../input/train_images/002039aaf8618627a0442d5e...,e98c873acc65946e,Korek Kuping LED untuk balita CherryBabyKidsSh...,349297863,[train_4196427721]
18,train_2985955659,../input/train_images/002f978c58a44a00aadfca71...,bf38f0e083d7c710,HnKfashion Sweater Hoodie WHO Printing BabyTer...,3415582503,[train_2985955659]


# title TFIDF

In [58]:
from PIL import Image

import torch

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer(stop_words=None, binary=True, max_features=20250)
text_embeddings = model.fit_transform(train.title).toarray()
print('text embeddings shape',text_embeddings.shape)

text_embeddings = torch.from_numpy(text_embeddings)
text_embeddings = text_embeddings.cuda()
del model
# preds = []
# CHUNK = 1024*4

# print('Finding similar titles...')
# CTS = len(train)//CHUNK
# if len(train)%CHUNK!=0: CTS += 1
# CTS_index = 0
# for j in range( CTS ):
    
#     a = j*CHUNK
#     b = (j+1)*CHUNK
#     b = min(b,len(train))
#     print('chunk',a,'to',b)
    
#     # COSINE SIMILARITY DISTANCE
#     # cts = np.dot( text_embeddings, text_embeddings[a:b].T).T
#     cts = torch.matmul(text_embeddings, text_embeddings[a:b].T).T
#     cts = cts.data.cpu().numpy()
#     print(cts.shape)
#     for k in range(b-a):
#         # IDX = np.where(cts[k,]>0.7)[0]
#         IDX = np.where(cts[k,]>0.7)[0]
#         o = train.iloc[IDX].posting_id.values
#         preds.append(o)
#         CTS_index += 1
# # del model, text_embeddings

# train['oof_text'] = preds
# # 仅考虑 TFIDF 的成绩
# if COMPUTE_CV:
#     train['f1'] = train.apply(getMetric('oof_text'),axis=1)
#     print('CV score for baseline =',train.f1.mean())
    
    
# 0.6137154152579091 0.7
# 0.6507316994356058 0.6

text embeddings shape (16, 141)


# image hash

In [59]:
# # 列出hash相同的所有id
# tmp = train.groupby('image_phash').posting_id.agg('unique').to_dict()
# train['oof_hash'] = train.image_phash.map(tmp)

In [60]:
# # 精确率“几乎” 100%的策略：图像 hash 相同的一组。并计算分数
# # 数据可能有噪声，所以无法保证100%精确率，另外此策略召唤率较低
# if COMPUTE_CV:
#     train['f1'] = train.apply(getMetric('oof_hash'),axis=1)
#     print('CV score for baseline =',train.f1.mean())

# image CNN

In [61]:


class ShopeeImageDataset(Dataset):
    def __init__(self, img_path, transform):
        self.img_path = img_path
        self.transform = transform
        
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        img = self.transform(img)
        return img
    
    def __len__(self):
        return len(self.img_path)

In [62]:
# 图像预处理 dataset 和 DataLoader
imagedataset = ShopeeImageDataset(
    train['image'].values,
    transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # resnet 18处理前需要按此Normalize
]))
    
imageloader = torch.utils.data.DataLoader(
    imagedataset,
    batch_size=40, shuffle=False, num_workers=2
)

In [63]:
class ShopeeImageEmbeddingNet(nn.Module):
    def __init__(self):
        super(ShopeeImageEmbeddingNet, self).__init__()
              
        model = models.resnet18(pretrained=False)
        # 作者评论： 在检索任务中，最大池化优于平均池化。
        model.avgpool = nn.AdaptiveMaxPool2d(output_size=(1, 1)) # 最后一层令大小为1*1，参数自适应
        model = nn.Sequential(*list(model.children())[:-1]) # remove the last fully-connected layer
        model.eval() # 不再改变权重
        self.model = model
        
    def forward(self, img):        
        out = self.model(img)
        return out

In [64]:
# 下载resnet18预训练模型
# !mkdir -p /root/.cache/torch/hub/checkpoints/
# !cp ../input/resnet18-f37072fd/resnet18-f37072fd.pth /root/.cache/torch/hub/checkpoints/

In [65]:
DEVICE = 'cuda'

imgmodel = ShopeeImageEmbeddingNet()
imgmodel = imgmodel.to(DEVICE)

imagefeat = []
with torch.no_grad():
    for data in tqdm_notebook(imageloader):
        data = data.to(DEVICE)
        feat = imgmodel(data)
        feat = feat.reshape(feat.shape[0], feat.shape[1])
        feat = feat.data.cpu().numpy()
        
        imagefeat.append(feat)

/home/cyl/miniconda3/envs/kaggle/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/cyl/miniconda3/envs/kaggle/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_4957/1849022496.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data in tqdm_notebook(imageloader):


  0%|          | 0/1 [00:00<?, ?it/s]

resnet18 以批次为单位训练
imagefeat 记录每一批次的结果，每一批次结果为一个维度为 (批大小)*512(通道) 的向量
接下来需要把所以批次合并，形成若干个512维向量，每个向量对应一张照片

In [66]:
from sklearn.preprocessing import normalize

# l2 norm to kill all the sim in 0-1
imagefeat = np.vstack(imagefeat) 
imagefeat = normalize(imagefeat) # 对每一个嵌入向量归一化 axis=1:normalize each sample

In [67]:
imagefeat = torch.from_numpy(imagefeat)
imagefeat = imagefeat.cuda()

In [68]:
preds = []
CHUNK = 1024*4


print('Finding similar images...')
CTS = len(imagefeat)//CHUNK
if len(imagefeat)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b, len(imagefeat))
    print('chunk',a,'to',b) # 对a到b的每一个数据计算它到其他数据的距离
    cts = torch.matmul(text_embeddings, text_embeddings[a:b].T).T
    cts = cts.data.cpu().numpy()
    distances = torch.matmul(imagefeat, imagefeat[a:b].T).T # 点乘计算余弦相似度，注意最后的转置
    distances = distances.data.cpu().numpy()
    # distances = np.dot(imagefeat[a:b,], imagefeat.T)
    
    for k in range(b-a):
        # IDX = cupy.where(distances[k,]>0.95)[0]
        similarity_score = (distances[k] / 0.84) ** 6 + (cts[k] / 0.84) ** 6
        IDX = np.where(similarity_score>1)[0][:] # 注意这个“距离”是余弦，角度越小余弦值越大，所以是越大越相似
        o = train.iloc[IDX].posting_id.values
#       o = train.iloc[cupy.asnumpy(IDX)].posting_id.values
        preds.append(o)

del imagefeat, imgmodel,text_embeddings

Finding similar images...
chunk 0 to 16


In [69]:
train['oof_cnn'] = preds
# 仅考虑 resnet 的成绩
if COMPUTE_CV:
    train['f1'] = train.apply(getMetric('oof_cnn'),axis=1)
    print('CV score for baseline =',train.f1.mean())
    
# 0.6527899883424048 0.95
# 0.6686372611222741 0.94
# 0.6762305764407363 0.93

CV score for baseline = 1.0


In [70]:
# 对每一个数据，取[tfidf匹配结果,cnn匹配结果,hash匹配结果]的交集
def combine_for_sub(row):
    x = np.concatenate([row.oof_text,row.oof_cnn, row.oof_hash])
    return ' '.join( np.unique(x) )

def combine_for_cv(row):
    x = np.concatenate([row.oof_text,row.oof_cnn, row.oof_hash])
    return np.unique(x)

In [71]:
# if COMPUTE_CV:
#     tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
#     train['target'] = train.label_group.map(tmp)
#     train['oof'] = train.apply(combine_for_cv,axis=1)
#     train['f1'] = train.apply(getMetric('oof'),axis=1)
#     print('CV Score =', train.f1.mean() )

# train['matches'] = train.apply(combine_for_sub,axis=1)
train['matches']=train['oof_cnn'] 
train['matches'].head()

2     [train_2288590299]
10    [train_1598329973]
14    [train_4287573913]
15    [train_4196427721]
18    [train_2985955659]
Name: matches, dtype: object

In [72]:
train[['posting_id','matches']].to_csv('submission.csv',index=False)
sub = pd.read_csv('submission.csv')
sub.head()

,posting_id,matches
0,train_2288590299,['train_2288590299']
1,train_1598329973,['train_1598329973']
2,train_4287573913,['train_4287573913']
3,train_4196427721,['train_4196427721']
4,train_2985955659,['train_2985955659']
